In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<font size="5"><b>Set GPU environment</font>

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
tf.__version__

'1.11.0'

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

<font size="5"><b>Data Preprocessing ( Cornell Movie-Dialogs )</font>

In [5]:
# load lines dictionary 
lines = open('hw13_data/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')

# load conversations
convs = open('hw13_data/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

print('\n'.join(lines[:3]))
print()
print('\n'.join(convs[:3]))

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']


In [6]:
map_convs = {}
for line in lines:
    map_convs[line.split("+++$+++")[0].strip()] = line.split("+++$+++")[-1].strip()

In [7]:
for conv in eval(convs[0].split("+++$+++")[-1]):
    print(map_convs[conv])

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?


In [8]:
# Sort the sentences into questions and answers
questions = []
answers = []

for conv in convs:
    if len(conv.split("+++$+++")) == 4:
        conv_list = eval(conv.split("+++$+++")[-1])
        for id,sent in enumerate(conv_list[:-1]):
            questions.append(map_convs[sent])
            answers.append(map_convs[conv_list[id+1]])

In [9]:
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/nlplab/yeema/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = ' '.join(nltk.sent_tokenize(text))
    return text

In [11]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [12]:
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

lengths = pd.DataFrame(lengths, columns=['lengths'])

In [13]:
lengths.describe()

,lengths
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [14]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [15]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
min_line_length = 1
max_line_length = 14
short_questions = []
short_answers = []

for question,answer in zip(clean_questions,clean_answers):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length and len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
            short_questions.append(question)
            short_answers.append(answer)

In [16]:
str_short_questions = ' '.join([s for s in short_questions])
str_short_answers = ' '.join([s for s in short_answers])

In [17]:
from collections import Counter

In [18]:
vocab = Counter(str_short_questions.split()+str_short_answers.split())

In [19]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
threshold = 8
questions_vocab_to_int = {}
answers_vocab_to_int = {}

for id, (word, count) in enumerate(vocab.items()):
    if count >= threshold:
        questions_vocab_to_int[word] = len(questions_vocab_to_int)
        answers_vocab_to_int[word] = len(answers_vocab_to_int)

In [20]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<END>','<UNK>','<BEG>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)

In [21]:
# inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [22]:
short_questions = ['<BEG> '+s+' <END>' for s in short_questions]
short_answers = ['<BEG> '+s+' <END>' for s in short_answers]
# short_answers = [s+' <END>' for s in short_answers]

In [23]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [29]:
# show vocab_to_int mapping
count = 0
for a,b,c,d in zip(questions_int , short_questions , answers_int,short_answers):
    count+=1
    print(' '.join([questions_int_to_vocab[aa] for aa in a]))
    print(b)
    print(' '.join([questions_int_to_vocab[aa] for aa in c]))
    print(d)
    print()
    if count>5:
          break

<BEG> well i thought we would start with <UNK> if that is okay with you <END>
<BEG> well i thought we would start with pronunciation if that is okay with you <END>
<BEG> not the hacking and <UNK> and spitting part please <END>
<BEG> not the hacking and gagging and spitting part  please <END>

<BEG> not the hacking and <UNK> and spitting part please <END>
<BEG> not the hacking and gagging and spitting part  please <END>
<BEG> okay then how about we try out some french <UNK> saturday night <END>
<BEG> okay then how about we try out some french cuisine  saturday  night <END>

<BEG> you are asking me out that is so cute that is your name again <END>
<BEG> you are asking me out  that is so cute that is your name again <END>
<BEG> forget it <END>
<BEG> forget it <END>

<BEG> no no it is my fault we did not have a proper introduction <END>
<BEG> no no it is my fault  we did not have a proper introduction <END>
<BEG> cameron <END>
<BEG> cameron <END>

<BEG> gosh if only we could find kat a boy

In [30]:
# filter sentences contain <UNK>
sorted_questions_clean = []
sorted_answers_clean = []

for i in range(len(short_questions)):
    if not(questions_vocab_to_int['<UNK>'] in questions_int[i] or answers_vocab_to_int['<UNK>'] in answers_int[i]): # remove '<UNK>' sentence
        sorted_questions_clean.append(questions_int[i])
        sorted_answers_clean.append(answers_int[i])

In [31]:
for i in range(10):
    print(' '.join([questions_int_to_vocab[sorted_questions_clean[i][j]]  for j in range(len(sorted_questions_clean[i]))]))
    print(' '.join([answers_int_to_vocab[sorted_answers_clean[i][j]]  for j in range(len(sorted_answers_clean[i]))]))
    print('')

<BEG> you are asking me out that is so cute that is your name again <END>
<BEG> forget it <END>

<BEG> no no it is my fault we did not have a proper introduction <END>
<BEG> cameron <END>

<BEG> gosh if only we could find kat a boyfriend <END>
<BEG> let me see what i can do <END>

<BEG> that is because it is such a nice one <END>
<BEG> forget french <END>

<BEG> there <END>
<BEG> where <END>

<BEG> you have my word as a gentleman <END>
<BEG> you are sweet <END>

<BEG> sure have <END>
<BEG> i really really really wanna go but i cannot not unless my sister goes <END>

<BEG> hi <END>
<BEG> looks like things worked out tonight huh <END>

<BEG> have fun tonight <END>
<BEG> tons <END>

<BEG> well no <END>
<BEG> then that is all you had to say <END>



In [32]:
len(sorted_questions_clean),len(sorted_answers_clean)

(86564, 86564)

In [33]:
ques_max_len = 0
ans_max_len = 0

for i in range(len(sorted_questions_clean)): # caculate max length
    ques_max_len = max(ques_max_len, len(sorted_questions_clean[i]))
    ans_max_len = max(ans_max_len, len(sorted_answers_clean[i]))

print(ques_max_len, ans_max_len)

16 16


<font size="5"><b>Training (step1: generate batch、step2: Seq2Seq)</font>

In [34]:
class BatchGenerator:
    def __init__(self, en_corpus, ch_corpus, en_pad, ch_pad, en_max_len, ch_max_len, batch_size):
#         assert len(en_corpus) == len(ch_corpus)
        
        batch_num = len(en_corpus)//batch_size
        n = batch_num*batch_size
        
        self.xs = [np.zeros(n, dtype=np.int32) for _ in range(en_max_len)] # encoder inputs
        self.ys = [np.zeros(n, dtype=np.int32) for _ in range(ch_max_len)] # decoder inputs
        self.gs = [np.zeros(n, dtype=np.int32) for _ in range(ch_max_len)] # decoder outputs
        self.ws = [np.zeros(n, dtype=np.float32) for _ in range(ch_max_len)] # decoder weight for loss caculation
        
        self.en_max_len = en_max_len
        self.ch_max_len = ch_max_len
        self.batch_size = batch_size
        
        for b in range(batch_num):
            for i in range(b*batch_size, (b+1)*batch_size):
                for j in range(len(en_corpus[i])-2):
                    self.xs[j][i] = en_corpus[i][j+1]
                for j in range(j+1, en_max_len):
                    self.xs[j][i] = en_pad
                if ch_corpus:
                    for j in range(len(ch_corpus[i])-1):
                        self.ys[j][i] = ch_corpus[i][j]
                        self.gs[j][i] = ch_corpus[i][j+1]
                        self.ws[j][i] = 1.0
                    for j in range(j+1, ch_max_len): # don't forget padding and let loss weight zero
                        self.ys[j][i] = ch_pad
                        self.gs[j][i] = ch_pad
                        self.ws[j][i] = 0.0
    
    def get(self, batch_id):
        x = [self.xs[i][batch_id*self.batch_size:(batch_id+1)*self.batch_size] for i in range(self.en_max_len)]
        y = [self.ys[i][batch_id*self.batch_size:(batch_id+1)*self.batch_size] for i in range(self.ch_max_len)]
        g = [self.gs[i][batch_id*self.batch_size:(batch_id+1)*self.batch_size] for i in range(self.ch_max_len)]
        w = [self.ws[i][batch_id*self.batch_size:(batch_id+1)*self.batch_size] for i in range(self.ch_max_len)]
        
        return x, y, g, w

batch = BatchGenerator(sorted_questions_clean, sorted_answers_clean, 
                       questions_vocab_to_int['<PAD>'], answers_vocab_to_int['<PAD>'], ques_max_len, ans_max_len, 4)

print("Encoder input")
print("Decoder input")
print("Decoder output")
print()

x, y, g, w = batch.get(0)
for i in range(4):
    print(' '.join([questions_int_to_vocab[x[j][i]] for j in range(ques_max_len)]))
    print(' '.join([answers_int_to_vocab[y[j][i]] for j in range(ans_max_len)]))
    print(' '.join([answers_int_to_vocab[g[j][i]] for j in range(ans_max_len)]))
    print('')

Encoder input
Decoder input
Decoder output

you are asking me out that is so cute that is your name again <PAD> <PAD>
<BEG> forget it <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
forget it <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

no no it is my fault we did not have a proper introduction <PAD> <PAD> <PAD>
<BEG> cameron <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
cameron <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

gosh if only we could find kat a boyfriend <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<BEG> let me see what i can do <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
let me see what i can do <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

that is because it is such a nice one <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
<BEG> forget french <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

In [44]:
class MachineTranslationSeq2Seq:
    def __init__(self, en_max_len, ch_max_len, en_size, ch_size):
        self.en_max_len = en_max_len
        self.ch_max_len = ch_max_len
        
        with tf.variable_scope('seq2seq_intput/output'):
            self.enc_inputs = [tf.placeholder(tf.int32, [None]) for i in range(en_max_len)] # time mojor feed
            self.dec_inputs = [tf.placeholder(tf.int32, [None]) for i in range(ch_max_len)]
            self.groundtruths = [tf.placeholder(tf.int32, [None]) for i in range(ch_max_len)]
            self.weights = [tf.placeholder(tf.float32, [None]) for i in range(ch_max_len)]
            
        with tf.variable_scope('seq2seq_rnn'): # training by teacher forcing
            self.out_cell = tf.contrib.rnn.LSTMCell(512)
            self.outputs, _ = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(self.enc_inputs, self.dec_inputs, 
                                                                                    self.out_cell, 
                                                                                    en_size, ch_size, 300)
        with tf.variable_scope('seq2seq_rnn', reuse=True): # predict by feeding previous
            self.pred_cell = tf.contrib.rnn.LSTMCell(512, reuse=True) # reuse cell for train and test
            self.predictions, _ = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(self.enc_inputs, self.dec_inputs, 
                                                                                        self.pred_cell, 
                                                                                        en_size, ch_size, 300, 
                                                                                        feed_previous=True)
        
        with tf.variable_scope('loss'):
            # caculate weighted loss
            self.loss = tf.reduce_mean(tf.contrib.legacy_seq2seq.sequence_loss_by_example(self.outputs, 
                                                                                          self.groundtruths, 
                                                                                          self.weights))
            self.optimizer = tf.train.AdamOptimizer(0.002).minimize(self.loss)
        
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config)
        self.saver = tf.train.Saver()
#         self.sess.run(tf.global_variables_initializer())
        self.saver.restore(self.sess, 'model/seq2seq/seq2seq_%d.ckpt'%(48))
    def train(self, x, y, g, w):
        fd = {}
        for i in range(self.en_max_len):
            fd[self.enc_inputs[i]] = x[i] # show how to feed a list
        
        for i in range(self.ch_max_len):
            fd[self.dec_inputs[i]] = y[i]
            fd[self.groundtruths[i]] = g[i]
            fd[self.weights[i]] = w[i]
        
        loss, _ = self.sess.run([self.loss, self.optimizer], fd)
        
        return loss

    def output(self, x, y):
        fd = {}
        for i in range(self.en_max_len):
            fd[self.enc_inputs[i]] = x[i]
        
        for i in range(self.ch_max_len):
            fd[self.dec_inputs[i]] = y[i]
        
        out = self.sess.run(self.outputs, fd)
        
        return out
    
    def predict(self, x, ch_beg):
        fd = {}
        for i in range(self.en_max_len):
            fd[self.enc_inputs[i]] = x[i]
        
        for i in range(self.ch_max_len): # when feed previous, the fist token should be '<BEG>', and others are useless
            if i==0:
                fd[self.dec_inputs[i]] = np.ones(y[i].shape, dtype=np.int32)*ch_beg
            else:
                fd[self.dec_inputs[i]] = np.zeros(y[i].shape, dtype=np.int32)
        
        pd = self.sess.run(self.predictions, fd)
        
        return pd
    
    def save(self, e):
        self.saver.save(self.sess, 'model/seq2seq/seq2seq_%d.ckpt'%(e+1))
    
    def restore(self, e):
        self.saver.restore(self.sess, 'model/seq2seq/seq2seq_%d.ckpt'%(e))

In [45]:
tf.reset_default_graph()
model = MachineTranslationSeq2Seq(ques_max_len, ans_max_len, len(questions_vocab_to_int), len(answers_vocab_to_int))

INFO:tensorflow:Restoring parameters from model/seq2seq/seq2seq_48.ckpt


In [46]:
EPOCHS = 50
BATCH_SIZE = 16
batch_num = len(sorted_questions_clean)//BATCH_SIZE
batch = BatchGenerator(sorted_questions_clean, sorted_answers_clean, 
                       questions_vocab_to_int['<PAD>'], answers_vocab_to_int['<PAD>'], ques_max_len, ans_max_len, BATCH_SIZE)

In [47]:
rec_loss = []
for e in range(48,EPOCHS):
    train_loss = 0
    
    for b in range(batch_num):
        x, y, g, w = batch.get(b)
        batch_loss = model.train(x, y, g, w)
        train_loss += batch_loss
#         break
    
    train_loss /= batch_num
    rec_loss.append(train_loss)
    print("epoch %d loss: %f" % (e, train_loss))
    
    model.save(e)
    
np.save('./model/seq2seq/rec_loss.npy', rec_loss)

epoch 48 loss: 2.503017
epoch 49 loss: 2.507307


epoch 0 loss: 4.530785<br>
epoch 1 loss: 4.076271<br>
epoch 2 loss: 3.795061<br>
epoch 4 loss: 3.437478<br>
epoch 5 loss: 3.287869<br>
epoch 6 loss: 3.165183<br>
epoch 7 loss: 3.079307<br>
epoch 8 loss: 3.003268<br>
epoch 9 loss: 2.934210<br>
epoch 10 loss: 2.879893<br>
epoch 11 loss: 2.849603<br>
epoch 12 loss: 2.800245<br>
epoch 13 loss: 2.775303<br>
epoch 14 loss: 2.760754<br>
epoch 15 loss: 2.726841<br>
epoch 16 loss: 2.719413<br>
epoch 17 loss: 2.753609<br>
epoch 18 loss: 2.757131<br>
epoch 19 loss: 2.678192<br>
epoch 20 loss: 2.669429<br>
epoch 21 loss: 2.669876<br>
epoch 22 loss: 2.663899<br>
epoch 23 loss: 2.657978<br>
epoch 24 loss: 2.640877<br>
epoch 25 loss: 2.613516<br>
epoch 26 loss: 2.598924<br>
epoch 27 loss: 2.643231<br>
epoch 28 loss: 2.588912<br>
epoch 29 loss: 2.576181<br>
epoch 30 loss: 2.566805<br>
epoch 31 loss: 2.555187<br>
epoch 32 loss: 2.559543<br>
epoch 33 loss: 2.559104<br>
epoch 34 loss: 2.522690<br>
epoch 35 loss: 2.532498<br>
epoch 36 loss: 2.547435<br>
epoch 37 loss: 2.539630<br>
epoch 38 loss: 2.564870<br>
epoch 39 loss: 2.534656<br>
epoch 40 loss: 2.546252<br>
epoch 41 loss: 2.530325<br>
epoch 42 loss: 2.548503<br>
epoch 43 loss: 2.547191<br>
epoch 44 loss: 2.533915<br>
epoch 45 loss: 2.512516<br>
epoch 46 loss: 2.540780<br>
epoch 47 loss: 2.494065<br>
epoch 48 loss: 2.503017<br>
epoch 49 loss: 2.507307

In [87]:
# restore model weights
model.restore(49)

INFO:tensorflow:Restoring parameters from model/seq2seq/seq2seq_49.ckpt


<font size="5"><b>test for five sentences:</b></font>

In [88]:
testInput = ["Hello.",
"How are you?",
"Where are you going?",
"You look great.",
"Good night."]
tests = []
for test in testInput:
    tmp = []
    for id  in ('<BEG> '+clean_text(test)+' <END>').split():
        tmp.append(questions_vocab_to_int[id])
    tests.append(tmp)

In [89]:
batch = BatchGenerator(tests, [], 
                       questions_vocab_to_int['<PAD>'], answers_vocab_to_int['<PAD>'], ques_max_len, ans_max_len, len(testInput))

In [90]:
xs,y,z,w = batch.get(0)

In [91]:
#  show test inputs' batch result
for i in range(len(testInput)):
    print(' '.join([questions_int_to_vocab[xs[j][i]] for j in range(ques_max_len)]))
    print('')

hello <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

how are you <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

where are you going <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

you look great <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

good night <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>



In [92]:
import random as rd
records = []

pd = model.predict(xs,answers_vocab_to_int['<PAD>'])
for j in range(len(pd[0])):
    ch_pd = [answers_int_to_vocab[ np.argmax(pd[i][j,:]) ] for i in range(ans_max_len)]
    if '<END>' in ch_pd:
        ch_pd = ch_pd[:ch_pd.index('<END>')]
    records.append(ch_pd)

In [93]:
for id,i in enumerate(records):
    print(testInput[id])
    print(i)
    print()

Hello.
['surprised']

How are you?
['fine']

Where are you going?
['home']

You look great.
['thanks']

Good night.
['good', 'boy']



<font size="5"><b>explain result:</b></font>

Hello.
['surprised']

一個壽星到了和朋友約的建築物，門是微打開的，於是他有點害怕的走進去走進黑暗的房間，說：Hello

朋友們立馬跳出來拿著插滿蠟燭的蛋糕，說：surprised

How are you?
['fine']

Where are you going?
['home']

You look great.
['thanks']

Good night.
['good', 'boy']

晚上媽媽坐在小男孩身邊哄他睡覺，小男孩快睡著時，輕聲跟媽媽說：good night

媽媽親吻他的額頭說：good boy